---

# 공시 정보

## 회사 고유번호 받아오기
- OPEN DART API에서 회사 고유번호 데이터 받아오기
- https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018

In [1]:
import requests
import zipfile
import io
import xml.etree.ElementTree as et
import pandas as pd

###################################################### 입력 ######################################################

crtfc_key = '571048d49fc6cd13fc20e20830a6c92df00f3abe' #API 인증키
corp_name = '좋은사람들' #회사 이름

##################################################################################################################

# API에서 데이터 받아온 뒤 압축 해제
params = {'crtfc_key': crtfc_key}
url = "https://opendart.fss.or.kr/api/corpCode.xml"
result = requests.get(url, params=params)
uzfile = zipfile.ZipFile(io.BytesIO(result.content))
final = uzfile.open(uzfile.namelist()[0])

# 데이터를 DataFrame 형태로 가공
root = et.fromstring(final.read().decode('utf-8'))
items_en = ["corp_code", "corp_name", "stock_code", "modify_date"]
items_kr = ["고유번호", "정식명칭", "종목코드", "최종변경일자"]
data = []

for child in root:
    if len(child.find('stock_code').text.strip()) > 1:
        data.append([])
        for item in items_en:
            data[-1].append(child.find(item).text)
            
corp_df = pd.DataFrame(data, columns = items_kr)

# 대상 회사의 고유번호를 corp_code 변수에 저장
corp_code = corp_df[corp_df['정식명칭'] == corp_name]['고유번호'].values[0]
print(f"'{corp_name}' 고유번호: {corp_code}")

'좋은사람들' 고유번호: 00205003


## 공시 목록 받아오기
- OPEN DART API에서 지정된 회사에 대한 전체 공시 목록 받아오기
- 공시 목록 필터링은 하기와 같음
    - 시기는 2017.01.01 ~ 2023.03.20
    - 최종보고서만을 대상으로 함
- https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019001

In [2]:
import urllib
from bs4 import BeautifulSoup as bs

###################################################### 입력 ######################################################

bgn_de = '20170101' #시작일
end_de = '20230320' #종료일
last_reprt_at = 'Y' #최종보고서 검색여부
page_count = '100' #페이지 별 건수

##################################################################################################################

list_df = pd.DataFrame()

for i in range(1, 6): #주먹구구식임 (나중에 다른 회사 => 수정해야 함)
    # API에서 데이터 받아온 뒤 파싱
    page_no = f'0000{i}'
    url = f"https://opendart.fss.or.kr/api/list.xml?crtfc_key={crtfc_key}&corp_code={corp_code}&bgn_de={bgn_de}&"
    url = url + f"end_de={end_de}&last_reprt_at={last_reprt_at}&page_no={page_no}&page_count={page_count}"
    result = urllib.request.urlopen(url).read()
    soup = bs(result, 'html.parser')
    te = soup.findAll("list")

    # 데이터를 DataFrame 형태로 가공
    items_en = ["report_nm", "rcept_no", "rcept_dt"]
    items_kr = ["보고서명", "접수번호", "접수일자"]

    for t in te:
        temp = pd.DataFrame(
            ([[t.report_nm.string, t.rcept_no.string, t.rcept_dt.string]]),
            columns=items_kr
        )
        list_df = pd.concat([list_df, temp])
            
list_df = list_df.reset_index(drop=True)
list_df

C:\Users\louis\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\louis\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup cons

,보고서명,접수번호,접수일자
0,주주총회소집공고,20230316001320,20230316
1,주주총회집중일개최사유신고,20230316901843,20230316
2,주주총회소집결의,20230316901834,20230316
3,매출액또는손익구조30%(대규모법인은15%)이상변동,20230316901504,20230316
4,대표이사변경,20230102900603,20230102
...,...,...,...
456,주주총회소집결의,20170303900308,20170303
457,의결권대리행사권유참고서류,20170303000219,20170303
458,감사보고서제출,20170302900360,20170302
459,매출액또는손익구조30%(대규모법인은15%)이상변동,20170209900673,20170209


## 공시 서류 원문 파일 다운로드
- OPEN DART API에서 하기 공시 보고서의 원본 파일 다운로드
    - '최대주주변경'
    - '대표이사변경'
    - '사업보고서 (yyyy.mm)'
    - '불성실공시법인지정 (사유)'
    - '감사보고서제출'
- https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019001

In [3]:
###################################################### 입력 ######################################################

target_names_kr = ['최대주주변경', '대표이사변경', '사업보고서', '불성실공시법인지정', '감사보고서제출']
target_names_en = ['juju', 'isa', '4up', 'bulsungsil', 'thankyou']

##################################################################################################################

for target_name_kr, target_name_en in zip(target_names_kr, target_names_en):
    # 받아올 공시 보고서를 이름으로 검색
    target_df = list_df[list_df['보고서명'].str.find(target_name_kr) != -1]
    errors = []

    # API에서 파일 받아온 뒤 압축 해제 및 저장
    for idx, rcept_no in enumerate(target_df.iloc[:]['접수번호']):
        try:
            url = f"https://opendart.fss.or.kr/api/document.xml?crtfc_key={crtfc_key}&rcept_no={rcept_no}"
            urllib.request.urlretrieve(url, "download.zip")
            with zipfile.ZipFile("download.zip", "r") as zip_ref:
                zip_ref.extractall(path=f"documents/{target_name_en}")
        except:
            errors.append([idx, rcept_no])
    
    # 파일 다운로드 실패한 목록을 txt 형식으로 저장
    if errors:
        with open(f"documents/{target_name_en}/errors.txt", "wt") as fw:
            fw.write(f"[{target_name_kr} 공시 원문 error]\n")
            for error in errors:
                fw.write(error[1])
                fw.write("\n")

---

# 최대주주 및 대표이사 변경 횟수

## 공시 원문 파일에서 데이터 추출
- 최대주주변경, 대표이사변경 공시 xml 파일로부터 최대주주 및 대표이사 변경 데이터 추출하기

In [4]:
from glob import glob

###################################################### 입력 ######################################################

targets_en = ['juju', 'isa']
targets_kr = {'juju': '최대주주', 'isa': '대표이사'}

##################################################################################################################

juju_isa_df = pd.DataFrame()
errors = []

for target in targets_en:
    what_i_want = []

    for file in glob(f'documents/{target}/*'):
        try:
            # xml 오픈 및 파싱 (인코딩 'cp949'로 시도 후 안 되면 'utf-8'로 시도)
            try:
                with open(file, 'r') as fr:
                    soup = bs(fr.read(), 'html.parser')
            except:
                with open(file, 'r', encoding='utf-8') as fr:
                    soup = bs(fr.read(), 'html.parser')
                    
            txts = []
            for span in soup.body.find_all("span"):
                txts.append(span.text)
                    
            # 저장된 파일 데이터에서 '변경 전 인물, 변경 후 인물, 변경일자' 추출
            before = []
            after = []
            change_date = []

            is_finished_a = False
            is_finished_b = False

            for i in range(len(txts)):
                if "변경전" in txts[i]:
                    if target == "juju": #'최대주주변경' 문서만 다른 형식
                        before.append(txts[i + 2])
                        after.append(txts[i + 9])
                    else:
                        before.append(txts[i + 1])
                        after.append(txts[i + 3])
                    is_finished_a = True

                if "변경일" in txts[i]:
                    change_date.append(txts[i + 1])
                    is_finished_b = True

                if is_finished_a and is_finished_b:
                    break

            # 추출한 정보를 저장
            try:
                what_i_want.append([target, before[-1], after[-1], change_date[-1]])
            except:
                print(f"error! {target}, {file}")

        except:
             errors.append([target, file])
            
    cols = ['분류', '변경전', '변경후', '변경일']
    juju_isa_df = pd.concat([juju_isa_df, pd.DataFrame(what_i_want, columns=cols).sort_values("변경일")])

juju_isa_df = juju_isa_df.reset_index(drop=True)

# 실패 사례가 발생한 경우 에러 메시지 출력
if errors:
    print("error!")
    for error in errors:
        print(f"  - 대상: {error[0]} | 파일명: {error[1]}")

juju_isa_df

,분류,변경전,변경후,변경일
0,juju,염덕희 외4,주식회사 컨텐츠제이케이,2018-04-25
1,juju,주식회사컨텐츠제이케이 외 1,제이에이치W투자조합,2018-10-29
2,juju,제이에이치W투자조합 외1,(주)제이에이치리소스,2021-02-25
3,juju,(주)제이에이치리소스,CREDIT SUISSE AG,2021-12-01
4,juju,CREDIT SUISSE AG,주식회사 우리인터텍스 외 2인,2022-10-05
5,isa,윤우환,조민,2018-03-09
6,isa,조민,이종현,2019-03-27
7,isa,이종현,-,2021-04-22
8,isa,김용석 대표집행임원,이종현 사내이사,2022-01-05
9,isa,이종현,최재영,2022-01-07


## 변경 횟수 확인
- 최대주주 및 대표이사 변경 횟수가 조건에 해당하는지 체크하기
    - 최대주주: 5년 내 2회 이상 변경
    - 대표이사: 5년 내 2회 이상 변경

In [5]:
import datetime

checks = {}

print("=" * 15 + " [ 최대주주, 대표이사 check ] " + "=" * 15 + "\n")

for target in targets_en:
    # 날짜 차이 계산
    target_df = juju_isa_df[juju_isa_df['분류'] == target]
    target_df['변경일'] = pd.to_datetime(target_df['변경일'].apply(str)) #NavigableString은 bs4의 형식 => str로 변경
    diff_between_days = (target_df['변경일'] - target_df['변경일'].shift(1)).fillna(pd.Timedelta(seconds=0))
    target_df['날짜 차이'] = diff_between_days
    
    # 5년 이내에 변경 2회 이상 했는지 확인
    before_date = target_df['변경일'].iloc[0]
    after_date = before_date.replace(year = before_date.year + 5)
    five_years = after_date - before_date

    is_checked = False
    for i in range(len(target_df)):
        how_much_changes = 1
        days = five_years
        for j in range(i + 1, len(target_df)):
            days -= target_df.iloc[j]['날짜 차이']
            if days >= datetime.timedelta(0):
                how_much_changes += 1
            if how_much_changes >= 2:
                print(f"  - 5년 내 {targets_kr[target]} 변경 2회 이상")
                checks[target] = True
                is_checked = True
                break
        if is_checked:
            break
            
print("\n" + "=" * 60)

=============== [ 최대주주, 대표이사 check ] ===============

  - 5년 내 최대주주 변경 2회 이상
  - 5년 내 대표이사 변경 2회 이상



C:\Users\louis\AppData\Local\Temp\ipykernel_1760\3944130475.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df['변경일'] = pd.to_datetime(target_df['변경일'].apply(str)) #NavigableString은 bs4의 형식 => str로 변경
C:\Users\louis\AppData\Local\Temp\ipykernel_1760\3944130475.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df['날짜 차이'] = diff_between_days
C:\Users\louis\AppData\Local\Temp\ipykernel_1760\3944130475.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

---

# 대여금 및 선급금의 증가

## 공시 원문 파일에서 데이터 추출
- 사업보고서 xml 파일로부터 대여금 및 선급금 추출하기
    - 대여금: '재무제표' 영역의 '대여금 추가'와 '단기대여금 추가' 항목 합산
    - 선급금: '재무제표 주석' 영역의 '선급금' 항목

In [6]:
# 정수형 데이터에 자릿수 콤마 넣어서 문자열로 반환
def comma(num):
    num = str(num)
    transformed_num = ""
    length = len(num)
    for n in num:
        if length % 3 == 0 and length != len(num):
            transformed_num += ","
        transformed_num += n
        length -= 1
    return transformed_num

bogo_df = pd.DataFrame()

for file in glob('documents/4up/*'):
    # xml 오픈 및 파싱 (인코딩 'cp949'로 시도 후 안 되면 'utf-8'로 시도)
    try:
        with open(file, 'r') as fr:
            soup = bs(fr.read(), 'html.parser')
    except:
        with open(file, 'r', encoding='utf-8') as fr:
            soup = bs(fr.read(), 'html.parser')
            
    if soup.find('document-name').text != "사업보고서":
        continue

    # '재무제표' 영역의 '대여금', '단기대여금' 항목 추출
    txts = []
    for section in soup.body.find_all("section-2"):
        if "4. 재무제표" in section.title.text:
            for tr in section.find_all("tr"):
                try:
                    if tr.td.text.strip('\n').strip('\u3000') == "대여금의 증가":
                        for td in tr.find_all("td"):
                            txts.append(td.text.strip('\n').strip('\u3000'))
                    if tr.td.text.strip('\n').strip('\u3000') == "단기대여금의 증가":
                        for td in tr.find_all("td"):
                            txts.append(td.text.strip('\n').strip('\u3000'))
                except:
                    None

    # 대여금 총액 계산 (대여금 + 단기대여금)
    possible = 0
    for i in range(len(txts)):
        if i % 4 == 1:
            try:
                possible += int(txts[i].strip('\n()').replace(',', '')) #대여?
            except:
                possible += 0
    
    # '재무제표 주석' 영역의 '선급금' 항목 추출
    txts = []
    for section in soup.body.find_all("section-2"):
        if "5. 재무제표 주석" in section.title.text:
            for tr in section.find_all("tr"):
                try:
                    if tr.td.text.strip('\n').strip('\u3000').strip('\t').strip(' ') == "선급금":
                        for td in tr.find_all("td"):
                            txts.append(td.text.strip('\n').strip('\u3000'))
                except:
                    None
    
    # 선급금 총액 계산
    adv_payments = int(txts[1].strip('\n()').replace(',', ''))
    
    # 사업보고서 표지의 '사업연도' 항목 추출
    txts = []
    for cover in soup.find_all("cover"):
        for tr in cover.find_all("tr"):
            try:
                if '사업연도' in tr.td.text:
                    for tu in tr.find_all("tu"):
                        year = tu.text.strip('\n').strip('\u3000')[:4]
            except:
                None

    # 사업연도 확인
    if not(2017 <= int(year) <= 2022):
        continue
        
    cols = ['사업연도', '대여금', '선급금']
    bogo_df = pd.concat([bogo_df, pd.DataFrame([[year, possible, adv_payments]], columns=cols)])
    
bogo_df = bogo_df.sort_values('사업연도').reset_index(drop=True)
bogo_df['대여금'] = bogo_df['대여금'].apply(comma)
bogo_df['선급금'] = bogo_df['선급금'].apply(comma)
bogo_df

C:\Users\louis\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
C:\Users\louis\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup cons

,사업연도,대여금,선급금
0,2017,"270,000,000","173,973,887"
1,2018,"70,000,000","125,919,581"
2,2019,"4,690,000,000","701,935,326"
3,2020,"12,562,235,700","189,250,000"
4,2021,"2,500,000,000","243,462,271"


---

# 불성실 공시법인 지정 여부

## 공시 원문 파일에서 데이터 추출
- 불성실공시법인지정 공시 xml 파일로부터 불성실 공시법인 지정여부 및 지정일 추출하기

In [7]:
bulsungsil_df = pd.DataFrame()

for file in glob("documents/bulsungsil/*"):
    # xml 오픈 및 파싱 (인코딩 'cp949'로 시도 후 안 되면 'utf-8'로 시도)
    try:
        with open(file, 'r') as fr:
            soup = bs(fr.read(), 'html.parser')
    except:
        with open(file, 'r', encoding='utf-8') as fr:
            soup = bs(fr.read(), 'html.parser')

    if "불성실공시법인지정예고" in soup.title.text:
        continue
        
    txts = []
    for span in soup.find_all('span'):
        txts.append(span.text)

    # 불성실 공시법인 지정일 항목 추출
    date = []
    for i in range(len(txts)):
        if txts[i] == "지정일":
            date.append(txts[i + 1])

    checks['bulsungsil'] = True
    cols = ['지정여부', "지정일"]
    bulsungsil_df = pd.concat([bulsungsil_df, pd.DataFrame([["True", date[0]]], columns=cols)])
    
bulsungsil_df = bulsungsil_df.sort_values("지정일").reset_index(drop=True)
bulsungsil_df

,지정여부,지정일
0,True,2018-12-21
1,True,2021-03-12


---

# 유상증자 횟수

## 증자(감자) 현황 데이터 받아오기
- OPEN DART API에서 증자 및 감자 현황 데이터 받아오기
- 데이터 필터링은 하기와 같음
    - 사업연도: 2021년
    - 보고서: 사업보고서
    - 발행 형태: 유상증자
- https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS002&apiId=2019004

In [8]:
###################################################### 입력 ######################################################

bsns_year = '2021' #사업연도
reprt_code = '11011' #보고서 코드(11011: 사업보고서)

##################################################################################################################

zungza_df = pd.DataFrame()
errors = []

# API에서 데이터 받아온 뒤 파싱
url = f"https://opendart.fss.or.kr/api/irdsSttus.xml?crtfc_key={crtfc_key}&corp_code={corp_code}&"
url = url + f"bsns_year={bsns_year}&reprt_code={reprt_code}"
result = urllib.request.urlopen(url).read()
xmlsoup = bs(result, 'html.parser')
te = xmlsoup.findAll("list")

# 데이터를 DataFrame 형태로 가공
items_en = ["isu_dcrs_de", "isu_dcrs_stle", "isu_dcrs_stock_knd",
            "isu_dcrs_qy", "isu_dcrs_mstvdv_fval_amount", "isu_dcrs_mstvdv_amount"]
items_kr = ["주식발행일자", "발행 형태", "발행 주식 종류",
            "발행 수량", "발행 주당 액면 가액", "발행 주당 가액"]

for t in te:
    try:
        temp = pd.DataFrame(
            ([[t.isu_dcrs_de.string, t.isu_dcrs_stle.string, t.isu_dcrs_stock_knd.string, t.isu_dcrs_qy.string,
               t.isu_dcrs_mstvdv_fval_amount.string, t.isu_dcrs_mstvdv_amount.string]]),
            columns=items_kr
        )
        if temp['발행 형태'].values[0].find('유상증자') != -1:
            zungza_df = pd.concat([zungza_df, temp])
    except:
        errors.append(t)
        
zungza_df = zungza_df.reset_index(drop=True)
zungza_df

C:\Users\louis\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


,주식발행일자,발행 형태,발행 주식 종류,발행 수량,발행 주당 액면 가액,발행 주당 가액
0,2018.10.30,유상증자(제3자배정),보통주,"3,495,688",500,"4,291"
1,2020.04.01,유상증자(주주배정),보통주,"20,000,000",500,"1,740"


## 유상증자 횟수 확인
- 유상증자 횟수가 조건에 해당하는지 체크하기
    - 5년 내 2회 이상

In [9]:
# 총액 계산하기
a = zungza_df['발행 수량'].str.replace(",", "").astype("int64")
b = zungza_df['발행 주당 가액'].str.replace(",", "").astype("int64")
zungza_df['총액'] = a.mul(b).astype(str)

# 자릿수 콤마 넣기
zungza_df['총액'] = zungza_df['총액'].apply(comma)

# 날짜 계산하기
zungza_df['주식발행일자'] = pd.to_datetime(zungza_df['주식발행일자'].apply(str)) #NavigableString은 bs4의 형식 => str로 변경
diff_between_days = (zungza_df['주식발행일자'] - zungza_df['주식발행일자'].shift(1)).fillna(pd.Timedelta(seconds=0))
zungza_df.insert(2, '날짜 차이', diff_between_days)

# 5년 이내에 유상증자를 2회 이상 했는지 알아보기
before_date = zungza_df['주식발행일자'][0]
after_date = before_date.replace(year = before_date.year + 5)
five_years = after_date - before_date

for i in range(len(zungza_df)):
    how_much_zungzas = 1
    days = five_years
    for j in range(i + 1, len(zungza_df)):
        days -= zungza_df.iloc[j]['날짜 차이']
        if days >= datetime.timedelta(0):
            how_much_zungzas += 1
        if how_much_zungzas >= 2:
            print("** 5년 내 유상증자 2회 이상 **")
            checks['zungza'] = True
            break

** 5년 내 유상증자 2회 이상 **


---

# 감사의견 거절 여부

## 공시 원문 파일에서 데이터 추출
- 감사보고서 xml 파일로부터 감사의견 거절 여부 추출하기

In [10]:
thankyou_df = pd.DataFrame()

for file in glob('documents/thankyou/*'):
    # 파일 열기 (인코딩 'cp949'로 시도 후 안 되면 'utf-8'로 시도)
    try:
        with open(file, 'r') as fr:
            soup = bs(fr.read(), 'html.parser')
    except:
        with open(file, 'r', encoding='utf-8') as fr:
            soup = bs(fr.read(), 'html.parser')

    txts = []
    for span in soup.body.find_all("span"):
        txts.append(span.text)

    # 해당 감사보고서에 감사의견 거절이 있지 확인
    gamsa_no = False
    gamsa_dates = []
    for i in range(len(txts)):
        if txts[i] == "-감사의견" and txts[i + 1] == "의견거절":
            gamsa_no = True
            checks['gamsa'] = True
        if "감사보고서 수령일자" in txts[i]:
            gamsa_dates.append(int(txts[i + 1][:4]) - 1)
    
    # 사업연도 확인
    year = min(gamsa_dates)
    if not(2017 <= int(year) <= 2022):
        continue
    
    cols = ['사업연도', '감사 의견거절 여부']
    thankyou_df = pd.concat([thankyou_df, pd.DataFrame([[str(year), gamsa_no]], columns=cols)])

thankyou_df = thankyou_df.sort_values('사업연도').reset_index(drop=True)
thankyou_df

,사업연도,감사 의견거절 여부
0,2017,False
1,2018,False
2,2019,False
3,2020,False
4,2021,True


# 결과

## 조건별 해당 여부

In [11]:
papago = {'juju': '최대주주변경 5년 내 2회 이상', 'isa': '대표이사변경 5년 내 2회 이상',
          'bulsungsil': '불성실 공시법인 지정', 'zungza': '유상증자 5년 내 2회 이상',
          'gamsa': '감사 의견거절'}
cols = ['항목', '해당 여부']

result_df = pd.DataFrame()
for keys, values in papago.items():
    try:
        result_df = pd.concat([result_df, pd.DataFrame([[values, checks[keys]]], columns=cols)])
    except:
        result_df = pd.concat([result_df, pd.DataFrame([[values, False]], columns=cols)])

result_df = result_df.reset_index(drop=True)
result_df

,항목,해당 여부
0,최대주주변경 5년 내 2회 이상,True
1,대표이사변경 5년 내 2회 이상,True
2,불성실 공시법인 지정,True
3,유상증자 5년 내 2회 이상,True
4,감사 의견거절,True


## 무자본 M&A 의심기업 여부

In [12]:
if result_df['해당 여부'].value_counts()[True] >= 4:
    print(f"""'{corp_name}'은(는) 무자본 M&A 의심기업입니다.""")
else:
    print(f"""'{corp_name}'은(는) 무자본 M&A 의심기업이 아닙니다.""")

'좋은사람들'은(는) 무자본 M&A 의심기업입니다.
